In [1]:
import os
import cv2
import numpy as np
import torch
from scipy.io import loadmat
from scipy.ndimage import gaussian_filter
from pathlib import Path
from tqdm import tqdm

DATA_ROOT = Path("Dataset/ShanghaiTech/part_A/train_data")
IMG_DIR = DATA_ROOT / "images"
GT_DIR  = DATA_ROOT / "ground-truth"

SAVE_DIR = Path("path/to/save_pt")

os.makedirs(SAVE_DIR, exist_ok=True)

TARGET_SIZE = (512, 512)
GT_SIZE = (64, 64)

def make_density_map(points, shape):
    h, w = shape
    density = np.zeros((h, w), np.float32)
    for x, y in points:
        if 0 <= int(x) < w and 0 <= int(y) < h:
            density[int(y), int(x)] = 1.0
    density = gaussian_filter(density, sigma=4)
    return density

for img_path in tqdm(sorted(IMG_DIR.glob("*.jpg"))):
    name = img_path.stem

    # load image
    img = cv2.imread(str(img_path))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, TARGET_SIZE).astype(np.float32) / 255.0
    img_tensor = torch.tensor(img).permute(2,0,1)

    # load GT
    mat = loadmat(str(GT_DIR / f"GT_{name}.mat"))
    points = mat["image_info"][0][0][0][0][0]  # (x, y)

    # full density map
    den = make_density_map(points, TARGET_SIZE)

    # downsample 1/8 for CSRNet
    den_small = cv2.resize(den, GT_SIZE) * 64
    den_tensor = torch.tensor(den_small).unsqueeze(0)

    # SAVE AS .PT FILES (image and density in `path/to/save_pt`)
    torch.save(img_tensor, SAVE_DIR / f"{name}_img.pt")
    torch.save(den_tensor, SAVE_DIR / f"{name}_den.pt")

print("✔ Preprocessing complete — .pt files saved.")


100%|██████████| 300/300 [00:11<00:00, 26.38it/s]

✔ Preprocessing complete — .pt files saved.
